# How to publish Pandas DataFrames to Tableau Server
<p>By <a href="https://www.linkedin.com/in/jmperafan/">Juan Manuel Perafan</a></p>

Tableau Server has many data governance features to promote exploration, collaboration, and security. <br>
Out of all of them, my favorite by far is the Data Server.

The Data Server is a powerful, but often underutilized, feature that allows you to publish certified data sources directly to Tableau Server. <br>
These data sources can be analyzed with Tableau Web Edit or directly in Tableau Desktop. There numerous benefits of using the Data Server:

- **User friendly**: Empowering your business users to use analyze this data Tableau without having to understand how to connect to a database, join tables, or write difficult calculations. 
- **Single Version of the Truth**: It removes the risk of ambiguity for important data versions of your data sources and ensures people use the right calculations and logic. 
- **Security**: It allows you to implement row-level security and allows you to monitor who sees what data.  

In my career, I have prepared a couple dozen datasources in Tableau Desktop for other analysts to explore in Tableau Server. This process might involve a lot of manual work and prone to errors. <br>
Additionally, there are some use cases (not many, but some) that might not be possible or extremely tedious in Tableau. Some examples include:

- The 'IN' operator from SQL
- Proper case for strings
- Regex lookup from right to left
- Prediction output from a Machine Learning Model
- Row_number or index without a Table Calculation
- Tokenizing strings
- Removing rows and columns if every value is NULL

But what if I told you that you can wrangle your data in Python and publish the output as a hyper extract in your Tableau Server? <br>
Follow the instructions if you want to learn how!

<a href="https://www.linkedin.com/feed/hashtag/juanalytics/">#juanalytics</a>

<p><img src="https://www.blastam.com/wp-content/uploads/tableau-python-forecast.png" alt="";></p>

## Loading the packages

There are multiple packages for you to manipulate your data in Python. <br>
They might different from use case to use case, but more often than not, it will involve one of these 4.

- Pandas
- Numpy
- Scipy
- Sklearn 

On top of any package you might use to preapare your data, there are 2 additionally packages you will need for this to work.

### pantab
This package uses the Tableau SDK to transform Pandas Dataframes into Hyper files. <br>
This package uses the Extract API 2.0 and installing it can get tricky. <a href="https://help.tableau.com/current/api/extract_api/en-us/Extract/extract_api_installing.html">Here is how to install it. </a>

### tableauserverclient
This package servers as a Python wrapper for the REST API. <br>
It allows to do multiple tasks, including publishing datasources to a server.

In [3]:
# The usual suspects: Pandas
import pandas as pd

# Pandleau is used transform a Pandas DataFrame into a Hyper file
# To use pantab you need to install the Extract API 2.0

import pantab #https://github.com/WillAyd/pantab

# With the tableauserverclient library you can do almost everything that you can do with the REST API, including: publish workbooks, create users, query sites, etc.
# For this specific tutorial, we will use TSC to publish our hyper file to a Tableau Server

import tableauserverclient as TSC #https://github.com/tableau/server-client-python

## Create a data source

For the sake of simplicity, I just created a very simple dataset for this tutorial. <br>
But it is 100% up to you! You can call data from many datasources and implement any logic. <br>
Machine Learning? Advanced Regex? APIs? Go crazy with that data!

In [4]:
# Create Pandas DataFrame or effectively load data and create it
df = pd.DataFrame(
    {'Name':['Juan', 'Carlos', 'Andres', 'Felipe']
     , 'Age':[20, 21, 19, 18]}
) 
  
# This is how the datasources we created looks like
df 

,Name,Age
0,Juan,20
1,Carlos,21
2,Andres,19
3,Felipe,18


## Transform your DataFrame into a Tableau Hyper file

Once you are done going crazy on your data, you will have to transform it into a Hyper file.<br>
Luckily for you, this is as simple and straight-forward as typing one line of code.  <br>
I saved a copy of this extract in a folder called datasets, but you can choose any place to store it.

In [5]:
import os
os.remove('datasets/extract.hyper') # Remove the file if it already exists

# Transform your data from a Pandas DataFrame to a Tableau Hyper Extract!
df_tableau = pantab.frame_to_hyper(df, 'datasets/extract.hyper')

## Sign in to your Tableau Server 

This part is trick to demo since most of these fields will depend on your own Tableau Server. <br>
In this first cell, we are signing into our own Tableau Server.<br>
I shouldn't have to point out that you should never hardcore your credentials. <br>
Consider using a solution like **os.environ['SECRET']** from the os library. 

In [ ]:
# Since this is not a real server, it will throw an error.
# Just change the parameters with your own information. 

# There are safer ways to pass your credentials... 
# But I will keep it simple for the sake of this tutorial
tableau_auth = TSC.TableauAuth('USERNAME', 'PASSWORD')

# create an instance for your server
server = TSC.Server('https://SERVER_URL')

# Select to which site. Leave empty if there is only one site
site_id=''

# call the sign-in method with the auth object
server.auth.sign_in(tableau_auth)

## Uploading the Extract to Tableau Server

This is the moment you have been waiting for! <b>
You should indicate

In [ ]:
# Create a new project id or select an existing one.
# 3a8b... is just an example. You should change it.
project_id = '3a8b6148-493c-11e6-a621-6f3499394a39'

# The path and name of the data source to publish
file_path = r'datasets/extract.hyper'
  
# Create a datasource with project id 3a8b...
new_datasource = TSC.DatasourceItem(project_id)

# You can publish the data source using this command
new_datasource = server.datasources.publish(
    
    # This is the object we created earlier with pandtab
     df_tableau
    
    # The path and name of the data source to publish
     , file_path
    
    # Mode, you can choose between 'CreateNew', 'OverWrite', or 'Append'
    , 'CreateNew'
)